**1. 준비 (GPU T4로 세팅)**

In [ ]:
!nvidia-smi -L || echo "No GPU detected (ok for quick tests)"
!pip -q install "transformers" "datasets==3.0.1" "torch==2.3.0+cu118" "torchvision==0.18.0+cu118" "torchaudio==2.3.0+cu118" "sentencepiece" \
                 "pandas" "numpy" "matplotlib" "tqdm" "evaluate" -f https://download.pytorch.org/whl/torch_stable.html

GPU 0: Tesla T4 (UUID: GPU-b5f75025-2920-91f7-cb5a-89907a00a11d)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.6/839.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2

**2. Github Link 추가하여 데이터 불러오기, 데이터 파일 명만 추가하여 파일 구분**

In [ ]:
import pandas as pd

# GitHub raw URL 목록
urls = [
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/1.R1%7E2.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/2.RTM.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/3.R3%7E5.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/4.PLAY-IN.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/5.PLAYOFF.csv"
]

dfs = []

for url in urls:
    # 파일 이름만 추출 (예: Round1_Bahrain.csv → Round1_Bahrain)
    filename = url.split("/")[-1].replace(".csv", "")

    # CSV 읽기
    df = pd.read_csv(url)

    dfs.append(df)

# 모든 데이터 합치기
df_all = pd.concat(dfs, ignore_index=True)

In [ ]:
df_all.head()

,time_text,timestamp,author,message,amount,format,date
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,NaN,R1~2,250604
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,NaN,R1~2,250604
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,NaN,R1~2,250604
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,NaN,R1~2,250604
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,NaN,R1~2,250604


**3. 데이터 컬럼 선택 및 클리닝**

클리닝
- 디시인사이드의 경우 "dc official App" 삭제
- 해시태그, 멘션, URL, 공백 삭제
- "ㅋㅋㅋ"나 "!!!" 등의 표현 수는 남겨두기

**추후논의하여 클리닝 수정 가능

In [ ]:
# 마이크로초 단위 timestamp를 datetime 객체로 변환
# unit='us'를 사용하여 마이크로초 단위임을 지정합니다.
df_all['datetime_utc'] = pd.to_datetime(df_all['timestamp'], unit='us', errors='coerce')

# 초 이하 단위 제거
df_all['datetime_utc'] = df_all['datetime_utc'].dt.floor('s')

# UTC 시간을 KST (UTC+9)로 변환
# 먼저 UTC 타임존 정보를 추가하고, 그 다음 KST로 변환합니다.
df_all['datetime_kst'] = df_all['datetime_utc'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')

# 초 이하 단위 제거
df_all['datetime_kst'] = df_all['datetime_kst'].dt.floor('s')

In [ ]:
#결과 확인
df_all.head()

,time_text,timestamp,author,message,amount,format,date,datetime_utc,datetime_kst
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,NaN,R1~2,250604,2025-06-04 05:20:50,2025-06-04 14:20:50+09:00
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,NaN,R1~2,250604,2025-06-04 05:32:59,2025-06-04 14:32:59+09:00
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,NaN,R1~2,250604,2025-06-04 05:42:30,2025-06-04 14:42:30+09:00
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,NaN,R1~2,250604,2025-06-04 05:49:39,2025-06-04 14:49:39+09:00
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,NaN,R1~2,250604,2025-06-04 05:49:57,2025-06-04 14:49:57+09:00


In [ ]:
import pandas as pd
import numpy as np
import re

# ---------------------------------------------------------------------
# 데이터 클리닝

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""

    # 1) URL 제거
    s = re.sub(r"http\S+|www\.\S+", " ", s)

    # 2) 멘션/해시태그 제거
    s = re.sub(r"[@#]\w+", " ", s)

    # 3) 반복 문자 축약 (ㅋㅋㅋㅋ -> ㅋㅋ, 아아아아 -> 아아, !!!!! -> !!)
    s = re.sub(r"([ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9!?.])\1{2,}", r"\1\1", s)

    #4) 이모지/특수문자 과도한 것 정리 (한글/영문/숫자/기본문장부호만 유지)
    s = re.sub(r"[^\w\s가-힣ㄱ-ㅎㅏ-ㅣ!?.]", " ", s, flags=re.UNICODE)

    # 생략 5) 팀명/드라이버 약칭 정규화 (원하면 계속 추가)
    # team_map = {
    #    "T1": "티원",
    #    "GEN": "젠지",
    #    "VER": "페르스타펜",
    #    "HAM": "해밀턴",
    #}
    #for short_name, full_name in team_map.items():
    #    s = re.sub(fr"\b{short_name}\b", full_name, s, flags=re.IGNORECASE)

    # 6) 공백 정리
    s = re.sub(r"\s+", " ", s).strip()
    return s

####클리닝은 논의 후 추가

# ---------------------------------------------------------------------
print("Before cleaning, rows:", len(df_all))
# 데이터프레임 컬럼 선택 (df_all에서 필요한 컬럼 선택)
df = df_all[["datetime_kst", "author", "message", "format"]].copy()

# 채팅 제한 봇 데이터 삭제 (유저id: UCSvjQBDgYDB5TGVmCZObcwA)
df = df[df['author'] != 'UCSvjQBDgYDB5TGVmCZObcwA'].copy()

# 클리닝
df["clean_text"] = df["message"].map(normalize_text)


# 빈 텍스트/공백만 남은 행 제거
df["clean_text"] = df["clean_text"].fillna("").str.strip()
df = df[df["clean_text"].astype(bool)].reset_index(drop=True)

print("After cleaning, rows:", len(df))


Before cleaning, rows: 1520294
After cleaning, rows: 1504522


In [ ]:
#데이터 확인
df.sample(5)

,datetime_kst,author,message,format,clean_text
1117765,2025-07-30 17:55:15+09:00,UCeUh4kvpMSsdML5E6ACADUQ,이즈리얼 아리는 도대체 왜 하는 거냐 ㅋㅋ,R3~5,이즈리얼 아리는 도대체 왜 하는 거냐 ㅋㅋ
273497,2025-05-09 17:00:11+09:00,UCa_2yKEiX6odZDjk1I5CCCQ,반반만 가다오,R1~2,반반만 가다오
1252996,2025-09-03 18:07:33+09:00,UC0LeGQ7aFzjG6uG0iEIAoeA,너 젠지 같았어!,PLAY-IN,너 젠지 같았어!
1136521,2025-07-27 15:40:55+09:00,UCxe72i9MO4dCnpVAZ5TSbFA,든프는 결국 꼴찌인건 변함없나,R3~5,든프는 결국 꼴찌인건 변함없나
442351,2025-04-20 15:11:52+09:00,UC5GRMdEAq2Sq_dUBiTvgFmQ,안딜이나 플레타나 거기서거기 같은데 왜 바꾸지,R1~2,안딜이나 플레타나 거기서거기 같은데 왜 바꾸지


**4. 파인튜닝을 위한 데이터셋 불러오기**

사용데이터: KMRE
- KMRE 한국어 영화 리뷰 댓글 감정 라벨링: https://github.com/passing2961/KMRE

In [ ]:
from numpy.random import PCG64
from datasets import load_dataset

import pickle as pc

with open('data/kmre_train', 'rb') as f:
    kmre_train_data = pc.load(f)
with open('data/kmre_dev', 'rb') as f:
    kmre_dev_data = pc.load(f)
with open('data/kmre_test', 'rb') as f:
    kmre_test_data = pc.load(f)

# 데이터 구조 확인 (예시: 첫 샘플 출력)
print(kmre_train_data[0])

['너~~~무 재미없어서 중간에 나왔습니다;;', np.int32(2)]


**5. KoBERT 모델 불러오기**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# KoBERT 모델 및 토크나이저 불러오기
MODEL_NAME = "monologg/kobert"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=6) # Adjusted num_labels to 6 based on KMRE dataset labels

print("KoBERT Tokenizer and Model loaded successfully.")

The repository monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert .
 You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KoBERT Tokenizer and Model loaded successfully.


**6. KoBERT Model Tokenizing & Training(Finetuning)**

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Convert the loaded KMRE data into pandas DataFrames
train_df = pd.DataFrame(kmre_train_data, columns=['text', 'label'])
dev_df = pd.DataFrame(kmre_dev_data, columns=['text', 'label'])
test_df = pd.DataFrame(kmre_test_data, columns=['text', 'label'])

# Convert pandas DataFrames to Hugging Face Datasets
# The KMRE dataset labels are 0-indexed, so we can use them directly.
datasets = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(dev_df),
    'test': Dataset.from_pandas(test_df)
})

print(datasets)
print(datasets['train'][0])

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# Apply tokenization to the datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Remove original text column and set format for PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

print(tokenized_datasets)
print(tokenized_datasets['train'][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 119995
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 29999
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 49997
    })
})
{'text': '너~~~무 재미없어서 중간에 나왔습니다;;', 'label': 2}


Map:   0%|          | 0/119995 [00:00<?, ? examples/s]

Map:   0%|          | 0/29999 [00:00<?, ? examples/s]

Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 119995
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 29999
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 49997
    })
})
{'label': tensor(2), 'input_ids': tensor([   2, 1457,  463,  463,  463, 6228, 3977, 6881, 6855, 6553, 4258, 6896,
        1395, 6701,  252,  252,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1, 

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
import evaluate

# Define metric function (accuracy for now)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Configure training arguments
training_args = TrainingArguments(
    output_dir="./results",           # output directory
    num_train_epochs=3,               # total number of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=16,    # batch size per device during evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir="./logs",             # directory for storing logs
    logging_steps=10,                 # log every X updates steps
    eval_strategy="epoch",            # evaluate each epoch
    save_strategy="epoch",            # save checkpoint each epoch
    load_best_model_at_end=True,      # load the best model when training ends
    metric_for_best_model="accuracy",  # use accuracy to evaluate the best model
    report_to="none"                  # Disable reporting to Weights & Biases
)

# Create a DataCollatorWithPadding using the tokenizer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator # Pass data_collator instead of tokenizer
)

# Start training
trainer.train()

/tmp/ipython-input-4027653509.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


**7. 감정분석**